<a href="https://colab.research.google.com/github/danielbehargithub/LinkedIn_Salary/blob/main/Profile_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install playwright
!playwright install


In [3]:
# Enter url, email and password
user_url = "https://www.linkedin.com/in/daniel-behar-168647280"
email = "daniel10behar@gmail.com"
password = "..."


In [4]:
import nest_asyncio
from playwright.async_api import async_playwright
import asyncio

async def scrape_user_profile(user_url, email, password):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)  # Running in headless mode
        page = await browser.new_page()

        print("Navigating to LinkedIn login page...")
        # Navigate to LinkedIn login page
        await page.goto("https://www.linkedin.com/login")
        await asyncio.sleep(3)

        # Perform login
        print("Logging into LinkedIn...")
        await page.fill('input[id="username"]', email)
        await page.fill('input[id="password"]', password)
        await page.click('button[type="submit"]')
        await asyncio.sleep(5)

        # Check for two-step verification
        if await page.is_visible('input[name="pin"]'):
            print("Two-step verification detected. Please enter the code sent to your email.")
            verification_code = input("Enter the verification code: ")
            await page.fill('input[name="pin"]', verification_code)
            await page.click('button[type="submit"]')
            await asyncio.sleep(3)

        # Relevant sections to scrape for the user
        profile_sections = [
            "/details/education/",
            "/details/skills/",
            "/details/experience/",
            # "/details/certifications/"
        ]

        scraped_data = {}  # Dictionary to store HTML content of each section

        # מספר ניסיונות להורדת HTML
        MAX_RETRIES = 3

        # Loop through the list of profile URLs
        for section in profile_sections:
            url = f"https://www.linkedin.com/in/...{section}"
            retries = 0

            while retries < MAX_RETRIES:
                try:
                    print(f"Navigating to section: {url} (Attempt {retries + 1})")

                    # Navigate to the section URL
                    await page.goto(url)
                    await asyncio.sleep(3)

                    # Get page title to verify if the correct page is loaded
                    page_title = await page.title()
                    section_name = section.strip("/").split("/")[-1]  # Extract section name
                    print(f"Page title: {page_title}")
                    print(f"Section name: {section_name}")

                    # Verify if the page is correct
                    if section_name.capitalize() not in page_title:
                        print(f"Page {url} did not load correctly. Retrying...")
                        retries += 1
                        await asyncio.sleep(5)  # Wait a bit before retrying
                        continue  # Retry the same section

                    # Save the HTML content of the section page
                    html = await page.content()
                    scraped_data[section_name] = html  # Store HTML content in dictionary
                    print(f"HTML content for {section_name} saved in memory.")

                    # Save the HTML file for debugging
                    filename = f"{section_name}.html"
                    with open(filename, "w", encoding="utf-8") as f:
                        f.write(html)
                    print(f"Saved HTML for {url} as {section_name}")

                    # Take a screenshot of the section page
                    screenshot_filename = f"{section_name}.png"
                    await page.screenshot(path=screenshot_filename, full_page=True)
                    print(f"Saved screenshot for {url} as {screenshot_filename}")

                    break  # If successful, exit the retry loop

                except Exception as e:
                    print(f"Failed to scrape {url}: {e}")
                    retries += 1
                    await asyncio.sleep(5)  # Wait a bit before retrying

            if retries == MAX_RETRIES:
                print(f"Failed to scrape {url} after {MAX_RETRIES} attempts. Skipping...")

        # Close the browser
        print("Closing the browser...")
        await browser.close()

        return scraped_data  # Return all scraped HTML as a dictionary



In [5]:
scraped_html_data = await scrape_user_profile(user_url, email, password)

from bs4 import BeautifulSoup

# Example HTML from the dictionary
html_skills = scraped_html_data.get("skills")
html_education = scraped_html_data.get("education")
html_experience = scraped_html_data.get("experience")

# Parse HTML
soup_skill = BeautifulSoup(html_skills, "html.parser")
soup_education = BeautifulSoup(html_education, "html.parser")
soup_experience = BeautifulSoup(html_experience, "html.parser")


Navigating to LinkedIn login page...
Logging into LinkedIn...
Navigating to section: https://www.linkedin.com/in/nisan-ital/details/education/ (Attempt 1)
Page title: (13) Education | Nisan Ital | LinkedIn
Section name: education
HTML content for education saved in memory.
Saved HTML for https://www.linkedin.com/in/nisan-ital/details/education/ as education
Saved screenshot for https://www.linkedin.com/in/nisan-ital/details/education/ as education.png
Navigating to section: https://www.linkedin.com/in/nisan-ital/details/skills/ (Attempt 1)
Page title: (13) Skills | Nisan Ital | LinkedIn
Section name: skills
HTML content for skills saved in memory.
Saved HTML for https://www.linkedin.com/in/nisan-ital/details/skills/ as skills
Saved screenshot for https://www.linkedin.com/in/nisan-ital/details/skills/ as skills.png
Navigating to section: https://www.linkedin.com/in/nisan-ital/details/experience/ (Attempt 1)
Page title: (13) LinkedIn
Section name: experience
Page https://www.linkedin.com

In [51]:
from re import sub
experience_section = soup_experience.find("main", {"aria-label": "Experience"})

records = experience_section.find_all("li", class_="pvs-list__paged-list-item artdeco-list__item pvs-list__item--line-separated pvs-list__item--one-column")
for record in records:
  sub_records = record.find_all("li", class_="pvs-list__paged-list-item pvs-list__item--one-column")
  # 1 position only in each company
  if len(sub_records) == 0:
      job_name_container = record.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold") or \
                            record.find("div", class_="display-flex align-items-center mr1 t-bold")
      job_name = (
          job_name_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
          if job_name_container else "N/A"
      )
    # משך העבודה וסוג העבודה
      job_place_and_type_container = record.find("span", class_="t-14 t-normal")
      if job_place_and_type_container:
          place_and_type = job_place_and_type_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
          if "·" in place_and_type:
              company_name, job_type = [part.strip() for part in place_and_type.split("·", 1)]
          else:
              company_name, job_type = "N/A", place_and_type
      else:
          company_name, job_type = "N/A", "N/A"

      job_info_container = record.find_all("span", class_="t-14 t-normal t-black--light")
      info = [span.find("span", {"aria-hidden": "true"}).get_text(strip=True) for span in job_info_container if span.find("span", {"aria-hidden": "true"})]
      job_duration = info[0] if len(info) > 0 else "N/A"
      job_location = info[1] if len(info) > 1 else "N/A"

      additional_content_container = record.find_all("div", class_="display-flex align-items-center t-14 t-normal t-black")
      additional_content_and_skills = [span.find("span", {"aria-hidden": "true"}).get_text(strip=True) for span in additional_content_container if span.find("span", {"aria-hidden": "true"})]
      additional_content = additional_content_and_skills[0] if len(additional_content_and_skills) > 0 else "N/A"

      skills = additional_content_and_skills[1] if len(additional_content_and_skills) > 1 else "N/A"
      skills = skills.replace("Skills:", "").strip()

      # הצגת התוצאות
      print(f"Job Name: {job_name}")
      print(f"Company: {company_name}")
      print(f"Job Type: {job_type}")
      print(f"Job Duration: {job_duration}")
      print(f"Job Location: {job_location}")
      print(f"Additional Content: {additional_content}")
      print(f"Skills: {skills}")
      print("-" * 50)
  else:
      title_container = record.find("div", class_="display-flex flex-row justify-space-between")
      company_name_container = title_container.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold") or \
                          title_container.find("div", class_="display-flex align-items-center mr1 t-bold")
      company_name = (
          company_name_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
          if company_name_container else "N/A"
      )
      duration_container = title_container.find("span", class_="t-14 t-normal")
      duration = duration_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)

      print(f"Company Name: {company_name}")
      print(f"Duration at company: {duration}")


      job_name_container = record.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold") or \
                            record.find("div", class_="display-flex align-items-center mr1 t-bold")
      job_name = (
          job_name_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
          if job_name_container else "N/A"
      )
    # משך העבודה וסוג העבודה
      job_place_and_type_container = record.find("span", class_="t-14 t-normal")
      if job_place_and_type_container:
          place_and_type = job_place_and_type_container.find("span", {"aria-hidden": "true"}).get_text(strip=True)
          if "·" in place_and_type:
              company_name, job_type = [part.strip() for part in place_and_type.split("·", 1)]
          else:
              company_name, job_type = "N/A", place_and_type
      else:
          company_name, job_type = "N/A", "N/A"

          job_info_container = record.find_all("span", class_="t-14 t-normal t-black--light")
          info = [span.find("span", {"aria-hidden": "true"}).get_text(strip=True) for span in job_info_container if span.find("span", {"aria-hidden": "true"})]
          job_duration = info[0] if len(info) > 0 else "N/A"
          job_location = info[1] if len(info) > 1 else "N/A"

          additional_content_container = record.find_all("div", class_="display-flex align-items-center t-14 t-normal t-black")
          additional_content_and_skills = [span.find("span", {"aria-hidden": "true"}).get_text(strip=True) for span in additional_content_container if span.find("span", {"aria-hidden": "true"})]
          additional_content = additional_content_and_skills[0] if len(additional_content_and_skills) > 0 else "N/A"

          skills = additional_content_and_skills[1] if len(additional_content_and_skills) > 1 else "N/A"
          skills = skills.replace("Skills:", "").strip()



          print(f"Job Name: {job_name}")
          print(f"Company: {company_name}")
          print(f"Job Type: {job_type}")
          print(f"Job Duration: {job_duration}")
          print(f"Job Location: {job_location}")
          print(f"Additional Content: {additional_content}")
          print(f"Skills: {skills}")
          print("-" * 50)


Company Name: Next Career
Duration at company: 2 yrs 1 mo
Job Name: Headhunter (via Next Career)
Company: Flex
Job Type: Contract
Job Duration: Apr 2024 - May 2024 · 2 mos
Job Location: Israel · Hybrid
Additional Content: Recruitment as a Service
Skills: N/A
--------------------------------------------------
Job Name: Headhunter (via Next Career)
Company: Puzzle Projects LTD
Job Type: Contract
Job Duration: Feb 2024 - Apr 2024 · 3 mos
Job Location: Israel · Remote
Additional Content: Recruitment as a service for airspace and defence company.
Skills: N/A
--------------------------------------------------
Job Name: Talent Acquisition Manager (via Next Career)
Company: Discount Bank בנק דיסקונט
Job Type: Contract
Job Duration: 2023 - 2024 · 1 yr
Job Location: Israel · Hybrid
Additional Content: N/A
Skills: N/A
--------------------------------------------------
Job Name: Global Sourcing Manager
Company: Lightricks
Job Type: Full-time
Job Duration: 2021 - 2023 · 2 yrs
Job Location: Israel
A

In [7]:
education_section = soup_education.find("main", {"aria-label": "Education"})

education_records = education_section.find_all("div", {"data-view-name": "profile-component-entity"})

for record in education_records:
    print("Education record:")

    # Institution name
    institution_container = record.find("div", class_="display-flex align-items-center mr1 hoverable-link-text t-bold")
    institution_name = institution_container.get_text(strip=True) if institution_container else "N/A"
    print(f"  Institution: {institution_name}")

    # Date range
    date_container = record.find("span", class_="pvs-entity__caption-wrapper")
    date_range = date_container.get_text(strip=True) if date_container else "N/A"
    print(f"  Date: {date_range}")

    # Additional description
    description_container = record.find("span", class_="t-14 t-normal")
    description_text = description_container.get_text(strip=True) if description_container else "N/A"
    print(f"  Description: {description_text}")

    # Skills
    skills_container = record.find("div", class_="display-flex align-items-center t-14 t-normal t-black")
    skills = []
    if skills_container:
        skills_span = skills_container.find("span", {"aria-hidden": "true"})
        if skills_span:
            skills_text = skills_span.get_text(strip=True)
            if "Skills:" in skills_text:
                skills_text = skills_text.replace("Skills:", "").strip()
            skills = [skill.strip() for skill in skills_text.split("·")]
    print(f"  Skills: {', '.join(skills)}")

    # Additional text
    additional_text_container = record.find("div", class_="inline-show-more-text--is-collapsed")
    additional_text = ""
    if additional_text_container:
        additional_span = additional_text_container.find("span", {"aria-hidden": "true"})
        if additional_span:
            additional_text = additional_span.get_text(" ", strip=True)
    print(f"  Additional Text: {additional_text}")


Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2019 - 2021
  Description: Master's degree, Organizational Behavior StudiesMaster's degree, Organizational Behavior Studies
  Skills: 
  Additional Text: Grade: 94
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, Human Resources Management and ServicesBachelor's degree, Human Resources Management and Services
  Skills: 
  Additional Text: Grade: 95
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, National Security Policy StudiesBachelor's degree, National Security Policy Studies
  Skills: 
  Additional Text: 


In [8]:
# Find all <a> tags with data-field="skill_page_skill_topic"
skill_links = soup_skill.find_all("a", {"data-field": "skill_page_skill_topic"})

# Extract skills
skills = []
for link in skill_links:
    # Find the <span> with aria-hidden="true" inside the link
    skill_span = link.find("span", {"aria-hidden": "true"})
    if skill_span:
        skill_text = skill_span.get_text(strip=True)
        skills.append(skill_text)

unique_skills = sorted(set(skills))
print("Unique skills:", unique_skills)


Unique skills: ['Analytical Skills', 'Candidate Management', 'Co-sourcing', 'Communication', 'Corporate Recruiting', 'Curriculum Vitae (CV)', 'English', 'Executive Search', 'Game Theory', 'Hiring', 'Human Resources (HR)', 'Interpersonal Skills', 'Interview Preparation', 'Leadership', 'LinkedIn', 'Microsoft Excel', 'Problem Solving', 'Recruiting', 'Recruitment-to-Recruitment', 'Resume Writing', 'Sales', 'Sourcing', 'Start-up Ventures', 'Statistical Data Analysis', 'Statistics', 'Strategic Sourcing', 'Strategy', 'Vacancies', 'Venture Capital']
